In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tokenizetrain/tokenizer.json
/kaggle/input/tokenizetrain/tokenizer_config.json
/kaggle/input/tokenizetrain/special_tokens_map.json
/kaggle/input/tokenizetrain/vocab.txt
/kaggle/input/berttrain/config.json
/kaggle/input/berttrain/pytorch_model.bin
/kaggle/input/kaggle-llm-science-exam/sample_submission.csv
/kaggle/input/kaggle-llm-science-exam/train.csv
/kaggle/input/kaggle-llm-science-exam/test.csv
/kaggle/input/notebookeb09198479/config.json
/kaggle/input/notebookeb09198479/__results__.html
/kaggle/input/notebookeb09198479/tokenizer.json
/kaggle/input/notebookeb09198479/tokenizer_config.json
/kaggle/input/notebookeb09198479/pytorch_model.bin
/kaggle/input/notebookeb09198479/submission.csv
/kaggle/input/notebookeb09198479/__notebook__.ipynb
/kaggle/input/notebookeb09198479/special_tokens_map.json
/kaggle/input/notebookeb09198479/__output__.json
/kaggle/input/notebookeb09198479/vocab.txt
/kaggle/input/notebookeb09198479/custom.css


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoModel, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForMultipleChoice

import pandas as pd
import torch
from transformers import BertForSequenceClassification
from transformers import BertForMultipleChoice, BertTokenizer
from transformers import BertForQuestionAnswering
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
df = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')


In [4]:
list_a = []
model_name = "allenai/scibert_scivocab_uncased"
model = AutoModelForMultipleChoice.from_pretrained('/kaggle/input/berttrain')
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/tokenizetrain')


for a in range(len(df['prompt'])):
    prompt = df.iloc[a]['prompt']
    choices = ["A", "B", "C", "D", "E"]
    choice_texts = df.iloc[a][choices].values.tolist()

    encoded = tokenizer([prompt]*5, choice_texts, padding=True, return_tensors='pt', truncation=True, max_length=512)
    input_ids = encoded['input_ids'].view(1, 5, -1)
    attention_mask = encoded['attention_mask'].view(1, 5, -1)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    # 上位k個の選択肢を取得
    k = 3  # 取得したい選択肢の数
    topk_values, topk_indices = torch.topk(logits, k)
    
    # logitsと選択肢を一緒にソート
    sorted_indices = torch.argsort(logits, descending=True)
    sorted_choices = [choices[index] for index in sorted_indices[0]]

# 上位k個の選択肢を取得
    top_choices = sorted_choices[:k]
    list_a.append(top_choices)
    

In [5]:
df['prediction']= list_a
submission = df[['id', 'prediction']]
submission['prediction'] = submission['prediction'].apply(lambda x: ' '.join(map(str, x)))
submission.to_csv('submission.csv', index=False)

pd.read_csv('submission.csv').head(20)


/tmp/ipykernel_20/1137045637.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['prediction'] = submission['prediction'].apply(lambda x: ' '.join(map(str, x)))


,id,prediction
0,0,D B A
1,1,A E C
2,2,A C B
3,3,C A B
4,4,D C A
5,5,B C A
6,6,A C D
7,7,D B A
8,8,C E D
9,9,A E B
